In [30]:
import pandas as pd

df = pd.read_csv('Lung Cancer_Ct.csv')
df


,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,...,MMX2_FAM,MMX2_JA270,MMX3_FAM,MMX3_HEX,MMX3_JA270,selected,base,addition,final_ct,mutation_detected
0,1,64,Male,Sweden,2016-04-05,Stage I,Yes,Passive Smoker,29.4,199,...,NaN,NaN,NaN,NaN,NaN,MMX1_JA270,31.001958,-0.3,31.2,S768I
1,2,50,Female,Netherlands,2023-04-20,Stage III,Yes,Passive Smoker,41.2,280,...,NaN,NaN,25.3,NaN,NaN,MMX3_FAM,25.194727,-0.3,25.3,L861Q
2,3,65,Female,Hungary,2023-04-05,Stage III,Yes,Former Smoker,44.0,268,...,NaN,26.0,NaN,NaN,NaN,MMX2_JA270,26.855725,-0.2,26.0,T790M
3,4,51,Female,Belgium,2016-02-05,Stage I,No,Passive Smoker,43.0,241,...,NaN,NaN,NaN,NaN,NaN,MMX1_JA270,30.513162,-0.3,28.9,S768I
4,5,37,Male,Luxembourg,2023-11-29,Stage I,No,Passive Smoker,19.7,178,...,29.9,NaN,NaN,NaN,NaN,MMX2_FAM,30.717370,-0.3,29.9,L858R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889995,889996,40,Male,Malta,2022-07-01,Stage IV,No,Passive Smoker,44.8,243,...,23.6,NaN,NaN,NaN,NaN,MMX2_FAM,23.779382,-0.3,23.6,L858R
889996,889997,62,Female,Cyprus,2015-09-27,Stage II,Yes,Former Smoker,21.6,240,...,NaN,NaN,NaN,NaN,NaN,MMX1_JA270,28.456438,-0.2,28.8,S768I
889997,889998,48,Female,Estonia,2016-03-27,Stage III,Yes,Never Smoked,38.6,242,...,NaN,NaN,25.6,NaN,NaN,MMX3_FAM,25.704700,0.0,25.6,L861Q
889998,889999,67,Female,Slovakia,2015-12-22,Stage IV,Yes,Former Smoker,18.6,194,...,NaN,NaN,NaN,NaN,NaN,MMX1_FAM,24.181000,-0.2,23.7,Ex19Del


### [ 파이썬 ]

In [31]:
select_id = int(input("id를 입력하시오: "))

row = df[df['id'] == select_id]

if not row.empty:
    data = row.iloc[0]
    
    print(f"id: {select_id}")
    print(f"Age: {data['age']}")
    print(f"Smoking status: {data['smoking_status']}")
    print(f"Asthma: {data['asthma']}")
    print(f"Cancer stage: {data['cancer_stage']}")
else:
    print("해당 id를 찾을 수 없습니다.")

id: 5
Age: 37
Smoking status: Passive Smoker
Asthma: 0
Cancer stage: Stage I


In [32]:
def classify_final_risk_group(row):
    # 1. 가족력 있음 (Yes)
    if row['family_history'] == 'Yes':
        if row['smoking_status'] in ['Current Smoker', 'Former Smoker']:
            return 'High Risk (유전+흡연)'
        elif row['smoking_status'] == 'Passive Smoker':
            return 'Medium Risk (유전+간접흡연)'
        else:
            return 'Genetic Risk (유전적 소인)'

    # 2. 가족력 없음 (No) -> 후천적/환경적 요인 분석
    else:
        # 2-1. 흡연
        if row['smoking_status'] in ['Current Smoker', 'Former Smoker']:
            return 'Smoker Risk (직접 흡연 원인)'
            
        # 2-2. 기저질환 있음
        elif row['other_cancer'] == 1 or row['cirrhosis'] == 1:          # 이것도 항목에서 뺐는데 이 명령어의 유무?
             return 'Vulnerable (신체 기저질환 취약)'
             
        # 2-3. [수정됨] 가족력X, 흡연X, 지병X -> 그렇다면 원인은 환경?
        # 요리사, 건설직, 화학공장 등 직업적 요인이나 환경 오염 의심
        else:
            return 'Environmental/Occupational Risk (환경/직업적 노출 의심)'
        

# smoking_status만 검색하는 창을 만들고 전체적 흐름으로 다 이어지게 만들지 이것만 따로 뺄지 회의하기!!

### [ 판다스 ]

#### [ smoking_status / Risk_Group ]

In [47]:
# id를 입력 받으면 smoking_status를 출력하는 명령문
select_id = 1   # 아이디를 검색

result_df = df.loc[df['id'] == select_id, ['id', 'smoking_status', 'asthma', 'cancer_stage']]  # 검색할 컬럼을 선택

if not result_df.empty:
    result_df['smoking_status']
    display(result_df) 
else:
    print("해당 id를 찾을 수 없습니다.")

,id,smoking_status,asthma,cancer_stage
0,1,Passive Smoker,0,Stage I


In [ ]:
# 가족력과 흡연 상태와 더불어 기저질환을 연관 지어서 그에 따른 그룹으로 분류 
# id를 검색 받으면 'Risk Group'을 출력

conditions = [
    (df['family_history'] == 'Yes') & (df['smoking_status'].isin(['Current Smoker', 'Former Smoker'])), # 1. High Risk
    (df['family_history'] == 'Yes') & (df['smoking_status'] == 'Passive Smoker'),                      # 2. Medium Risk
    (df['family_history'] == 'Yes'),                                                                    # 3. Genetic Risk
    (df['family_history'] == 'No') & (df['smoking_status'].isin(['Current Smoker', 'Former Smoker'])), # 4. Smoker Risk
    (df['family_history'] == 'No') & ((df['other_cancer'] == 1) | (df['cirrhosis'] == 1))              # 5. Vulnerable
]

choices = [
    'High Risk (유전+흡연)',
    'Medium Risk (유전+간접흡연)',
    'Genetic Risk (유전적 소인)',
    'Smoker Risk (직접 흡연 원인)',
    'Vulnerable (신체 기저질환 취약)'
]


df['Risk_Group'] = np.select(conditions, choices, default='Environmental/Occupational Risk (환경/직업적 노출 의심)')

select_id = int(input())


result_df = df.loc[df['id'] == select_id, ['id', 'Risk_Group']]

if not result_df.empty:
    display(result_df)
else:
    print(f"해당 ID ({select_id})를 데이터에서 찾을 수 없습니다.")

▶ ID 2의 분석 결과입니다.


,id,Risk_Group
1,2,Medium Risk (유전+간접흡연)


In [ ]:
# 'Risk_Group' 내용이 잘 들어갔는지 확인
print(df['Risk_Group'].unique())

['Medium Risk (유전+간접흡연)' 'High Risk (유전+흡연)'
 'Environmental/Occupational Risk (환경/직업적 노출 의심)'
 'Vulnerable (신체 기저질환 취약)' 'Smoker Risk (직접 흡연 원인)'
 'Genetic Risk (유전적 소인)']


In [ ]:
# 리스크를 입력 받으면 그에 해당하는 id 출력
target_group = 'High Risk (유전+흡연)'
target_ids = df.loc[df['Risk_Group'] == target_group, ['id']]
display(target_ids)

,id
2,3
7,8
8,9
9,10
11,12
...,...
889982,889983
889994,889995
889996,889997
889998,889999


#### [ treatment_type ]

In [54]:
# id를 입력 받으면 treatment_type과 그와 연관 된다고 생각하는 컬럼 출력하는 명령문
select_id = 1   # 아이디를 검색

result_df = df.loc[df['id'] == select_id, ['id', 'treatment_type', 'age', 'cancer_stage','hypertension', 'asthma', 'cirrhosis', 'other_cancer']]  # 검색할 컬럼을 선택

if not result_df.empty:
    result_df['treatment_type']
    display(result_df) 
else:
    print("해당 id를 찾을 수 없습니다.")

,id,treatment_type,age,cancer_stage,hypertension,asthma,cirrhosis,other_cancer
0,1,Chemotherapy,64,Stage I,0,0,1,0


#### [ survived ]

In [55]:
# id를 입력 받으면 survived과 그와 연관 된다고 생각하는 컬럼 출력하는 명령문
select_id = 1   # 아이디를 검색

result_df = df.loc[df['id'] == select_id, ['id', 'survived', 'age','cancer_stage', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer', 'treatment_type']]  # 검색할 컬럼을 선택

if not result_df.empty:
    result_df['survived']
    display(result_df) 
else:
    print("해당 id를 찾을 수 없습니다.")

,id,survived,age,cancer_stage,hypertension,asthma,cirrhosis,other_cancer,treatment_type
0,1,0,64,Stage I,0,0,1,0,Chemotherapy
